In [1]:
from deta import Deta
import pandas as pd
import json
from pandas import json_normalize
from collections import Counter

In [2]:
DETA_KEY = "c0jo61nr_BhSm5qHprUP75vRSdEmumYfoS1KMCtQW"

# Initialize with a project key
deta = Deta(DETA_KEY)

# This is how to create/connect a database
db = deta.Base("utilities_reports")

In [3]:
def insert_period(period, e_usages, e_costs, w_usages, w_costs, digi_zahirs, digi_anis, 
                  streamyxs, comment):
    """Returns the user on a successful user creation, otherwise raises and error"""
    return db.put({"key": period, "e_usage": e_usages, "e_cost": e_costs, 
                   "w_usage": w_usages, "w_cost": w_costs, "digi_zahir": digi_zahirs, 
                   "digi_ani": digi_anis, "streamyx": streamyxs, "comment": comment})

def fetch_all_periods():
    """Returns a dict of all periods"""
    res = db.fetch()
    return res.items


def get_period(period):
    """If not found, the function will return None"""
    return db.get(period)

In [19]:
# Example Data
period = "2020_December"
comment = ""
e_usages = {"kWh": 341}
e_costs = {"RM_e": 84.8}
w_usages = {"m3": 27}
w_costs = {"RM_w": 18.6}
digi_zahirs = {"RM_d1": 0}
digi_anis = {"RM_d2": 0}
streamyxs = {"RM_s": 147.35}
insert_period(period, e_usages, e_costs, w_usages, w_costs, digi_zahirs, digi_anis, streamyxs, comment)

{'comment': '',
 'digi_ani': {'RM_d2': 0},
 'digi_zahir': {'RM_d1': 0},
 'e_cost': {'RM_e': 84.8},
 'e_usage': {'kWh': 341},
 'key': '2020_December',
 'streamyx': {'RM_s': 147.35},
 'w_cost': {'RM_w': 18.6},
 'w_usage': {'m3': 27}}

In [5]:
r = db.get('2022_January').values()
r

dict_values(['Test', {'RM_d2': 100}, {'RM_d1': 100}, {'RM_e': 100}, {'kWh': 100}, '2022_January', {'RM_s': 100}, {'RM_w': 100}, {'m3': 100}])

In [6]:
#total_e_usage = sum([sum(entry['e_usage'].values()) for entry in entries])
#total_e_cost = sum([sum(entry['e_cost'].values()) for entry in entries])
#total_w_usage = sum([sum(entry['w_usage'].values()) for entry in entries])
#total_w_cost = sum([sum(entry['w_cost'].values()) for entry in entries])

In [18]:
entries = fetch_all_periods()
df = json.dumps(entries)
df = pd.read_json(df)
df['key'] = df['key'].str.replace('_',' ')
df_1 = df['key'].str.split(' ', 1, expand=True)
df = pd.concat([df,df_1],axis=1)
df = df[['digi_ani','digi_zahir','e_cost','e_usage','streamyx','w_cost','w_usage',0,1]]
df = df.rename(columns={0:'year',1:'month'})
df_2023 = df[df.year == '2023']
df_2022 = df[df.year == '2022']
df_2022

,digi_ani,digi_zahir,e_cost,e_usage,streamyx,w_cost,w_usage,year,month
0,{'RM_d2': 123},{'RM_d1': 123},{'RM_e': 123},{'kWh': 123},{'RM_s': 123},{'RM_w': 123},{'m3': 123},2022,February
1,{'RM_d2': 100},{'RM_d1': 100},{'RM_e': 100},{'kWh': 100},{'RM_s': 100},{'RM_w': 100},{'m3': 100},2022,January


In [27]:
df_ecost_2022 = df_2022['e_cost'].map(Counter).groupby(df['year']).sum().reset_index()

#df_ecost_2022 = df_ecost_2022.to_dict()
s = df_ecost_2022['e_cost'].apply(lambda x: x.get('RM_e')).dropna()
s

0    223
Name: e_cost, dtype: int64

In [29]:
df_2022

,digi_ani,digi_zahir,e_cost,e_usage,streamyx,w_cost,w_usage,year,month
0,{'RM_d2': 123},{'RM_d1': 123},{'RM_e': 123},{'kWh': 123},{'RM_s': 123},{'RM_w': 123},{'m3': 123},2022,February
1,{'RM_d2': 100},{'RM_d1': 100},{'RM_e': 100},{'kWh': 100},{'RM_s': 100},{'RM_w': 100},{'m3': 100},2022,January
